import necesasry library

In [1]:
!pip install pyspark

In [2]:
from pyspark import SparkContext

In [3]:
from pyspark.sql import SparkSession

# Get or create SparkSession
spark = SparkSession.builder.getOrCreate()

# Rest of the code...


In [4]:

from __future__ import print_function
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
data = spark.read.csv("Admission_Predict.csv",header = True)

In [6]:
import pandas as pd

In [7]:
type(data)

pyspark.sql.dataframe.DataFrame

In [8]:
data.show()

+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
|         1|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|
|         2|      324|        107|                4|  4| 4.5|8.87|       1|            0.76|
|         3|      316|        104|                3|  3| 3.5|   8|       1|            0.72|
|         4|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|
|         5|      314|        103|                2|  2|   3|8.21|       0|            0.65|
|         6|      330|        115|                5|4.5|   3|9.34|       1|             0.9|
|         7|      321|        109|                3|  3|   4| 8.2|       1|            0.75|
|         8|      308|        101|                2|  3|   4| 7.9|    

Remove serial no for further work

In [9]:
data = data.drop("Serial No.")
data.printSchema()


root
 |-- GRE Score: string (nullable = true)
 |-- TOEFL Score: string (nullable = true)
 |-- University Rating: string (nullable = true)
 |-- SOP: string (nullable = true)
 |-- LOR : string (nullable = true)
 |-- CGPA: string (nullable = true)
 |-- Research: string (nullable = true)
 |-- Chance of Admit : string (nullable = true)



In [10]:
data.show(50)

+---------+-----------+-----------------+---+----+----+--------+----------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+---------+-----------+-----------------+---+----+----+--------+----------------+
|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|
|      324|        107|                4|  4| 4.5|8.87|       1|            0.76|
|      316|        104|                3|  3| 3.5|   8|       1|            0.72|
|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|
|      314|        103|                2|  2|   3|8.21|       0|            0.65|
|      330|        115|                5|4.5|   3|9.34|       1|             0.9|
|      321|        109|                3|  3|   4| 8.2|       1|            0.75|
|      308|        101|                2|  3|   4| 7.9|       0|            0.68|
|      302|        102|                1|  2| 1.5|   8|       0|             0.5|
|      323|     

In [11]:
data.printSchema()

root
 |-- GRE Score: string (nullable = true)
 |-- TOEFL Score: string (nullable = true)
 |-- University Rating: string (nullable = true)
 |-- SOP: string (nullable = true)
 |-- LOR : string (nullable = true)
 |-- CGPA: string (nullable = true)
 |-- Research: string (nullable = true)
 |-- Chance of Admit : string (nullable = true)



In [12]:
data.columns

['GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR ',
 'CGPA',
 'Research',
 'Chance of Admit ']

In [13]:
from pyspark.sql.functions import col

new_data = data.withColumn("GRE Score", col("GRE Score").cast("float")) \
               .withColumn("TOEFL Score", col("TOEFL Score").cast("float")) \
               .withColumn("University Rating", col("University Rating").cast("float")) \
               .withColumn("SOP", col("SOP").cast("float")) \
               .withColumn("LOR ", col("LOR ").cast("float")) \
               .withColumn("CGPA", col("CGPA").cast("float")) \
               .withColumn("Research", col("Research").cast("float")) \
               .withColumn("Chance of Admit ", col("Chance of Admit ").cast("float"))

new_data.printSchema()



root
 |-- GRE Score: float (nullable = true)
 |-- TOEFL Score: float (nullable = true)
 |-- University Rating: float (nullable = true)
 |-- SOP: float (nullable = true)
 |-- LOR : float (nullable = true)
 |-- CGPA: float (nullable = true)
 |-- Research: float (nullable = true)
 |-- Chance of Admit : float (nullable = true)



In [14]:
new_data.show()

+---------+-----------+-----------------+---+----+----+--------+----------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+---------+-----------+-----------------+---+----+----+--------+----------------+
|    337.0|      118.0|              4.0|4.5| 4.5|9.65|     1.0|            0.92|
|    324.0|      107.0|              4.0|4.0| 4.5|8.87|     1.0|            0.76|
|    316.0|      104.0|              3.0|3.0| 3.5| 8.0|     1.0|            0.72|
|    322.0|      110.0|              3.0|3.5| 2.5|8.67|     1.0|             0.8|
|    314.0|      103.0|              2.0|2.0| 3.0|8.21|     0.0|            0.65|
|    330.0|      115.0|              5.0|4.5| 3.0|9.34|     1.0|             0.9|
|    321.0|      109.0|              3.0|3.0| 4.0| 8.2|     1.0|            0.75|
|    308.0|      101.0|              2.0|3.0| 4.0| 7.9|     0.0|            0.68|
|    302.0|      102.0|              1.0|2.0| 1.5| 8.0|     0.0|             0.5|
|    323.0|     

In [15]:
from pyspark.sql.functions import col, count, isnan, when


In [16]:
for c in new_data.columns:
    print(c)

GRE Score
TOEFL Score
University Rating
SOP
LOR 
CGPA
Research
Chance of Admit 


In [17]:
### Checking for null ir nan type values in out columns
new_data.select([count(when(col(c).isNull(), c)).alias(c) for c in new_data.columns]).show()

+---------+-----------+-----------------+---+----+----+--------+----------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+---------+-----------+-----------------+---+----+----+--------+----------------+
|        0|          0|                0|  0|   0|   0|       0|               0|
+---------+-----------+-----------------+---+----+----+--------+----------------+



In [18]:
new_data

DataFrame[GRE Score: float, TOEFL Score: float, University Rating: float, SOP: float, LOR : float, CGPA: float, Research: float, Chance of Admit : float]

In [19]:
features = new_data.drop('Chance of Admit ')


In [20]:
features

DataFrame[GRE Score: float, TOEFL Score: float, University Rating: float, SOP: float, LOR : float, CGPA: float, Research: float]

In [21]:
features.show(2)

+---------+-----------+-----------------+---+----+----+--------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|
+---------+-----------+-----------------+---+----+----+--------+
|    337.0|      118.0|              4.0|4.5| 4.5|9.65|     1.0|
|    324.0|      107.0|              4.0|4.0| 4.5|8.87|     1.0|
+---------+-----------+-----------------+---+----+----+--------+
only showing top 2 rows



In [22]:
#let's assemble our features together using vectorAssembler
assembler = VectorAssembler( inputCols=features.columns,outputCol="features")

In [23]:
output = assembler.transform(new_data)


In [24]:
output

DataFrame[GRE Score: float, TOEFL Score: float, University Rating: float, SOP: float, LOR : float, CGPA: float, Research: float, Chance of Admit : float, features: vector]

implement linear regression

In [26]:
output= output.select("features", "Chance of Admit ")


In [27]:
output = assembler.transform(new_data)

In [28]:
output

DataFrame[GRE Score: float, TOEFL Score: float, University Rating: float, SOP: float, LOR : float, CGPA: float, Research: float, Chance of Admit : float, features: vector]

In [30]:
output= output.select("features", "Chance of Admit ")


In [32]:
train_df,test_df = output.randomSplit([0.7, 0.3])

In [33]:
train_df.show()
test_df.show()

+--------------------+----------------+
|            features|Chance of Admit |
+--------------------+----------------+
|[290.0,104.0,4.0,...|            0.45|
|[293.0,97.0,2.0,2...|            0.64|
|[294.0,95.0,1.0,1...|            0.49|
|[295.0,99.0,2.0,2...|            0.57|
|[296.0,95.0,2.0,3...|            0.44|
|[296.0,97.0,2.0,1...|            0.49|
|[296.0,99.0,2.0,2...|            0.61|
|[296.0,101.0,1.0,...|             0.6|
|[297.0,96.0,2.0,2...|            0.43|
|[297.0,96.0,2.0,2...|            0.34|
|[297.0,98.0,2.0,2...|            0.59|
|[297.0,100.0,1.0,...|            0.52|
|[298.0,92.0,1.0,2...|            0.51|
|[298.0,98.0,2.0,1...|            0.44|
|[298.0,98.0,2.0,4...|            0.34|
|[298.0,99.0,2.0,4...|            0.46|
|[298.0,105.0,3.0,...|            0.69|
|[299.0,94.0,1.0,1...|            0.42|
|[299.0,96.0,2.0,1...|            0.54|
|[299.0,97.0,3.0,5...|            0.38|
+--------------------+----------------+
only showing top 20 rows

+-------------

In [34]:
lin_reg = LinearRegression(featuresCol = 'features', labelCol='Chance of Admit ')
linear_model = lin_reg.fit(train_df)

23/06/23 16:00:00 WARN Instrumentation: [ac170755] regParam is zero, which might cause numerical instability and overfitting.
23/06/23 16:00:01 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/06/23 16:00:01 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/06/23 16:00:01 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/06/23 16:00:01 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [35]:

print("Coefficients: " + str(linear_model.coefficients))
print("Intercept: " + str(linear_model.intercept))
     

Coefficients: [0.0019815674751623405,0.0038676717057704495,0.004665619553436757,-0.0043041063174017085,0.023785434284714906,0.11936720304496554,0.017113173430002016]
Intercept: -1.435505219014181


In [36]:

trainSummary = linear_model.summary
print("RMSE: %f" % trainSummary.rootMeanSquaredError)
print("r2: %f" % trainSummary.r2)

RMSE: 0.061823
r2: 0.811308


In [38]:

# prediction

predictions = linear_model.transform(test_df)
predictions.select("prediction","Chance of Admit ","features").show()

+-------------------+----------------+--------------------+
|         prediction|Chance of Admit |            features|
+-------------------+----------------+--------------------+
|0.47411289619640007|            0.47|[290.0,100.0,1.0,...|
| 0.4310920463151504|            0.46|[294.0,93.0,1.0,1...|
| 0.4118227694396892|            0.46|[295.0,93.0,1.0,2...|
| 0.4469549066769103|            0.47|[295.0,96.0,2.0,1...|
| 0.5240601021949938|            0.69|[295.0,101.0,2.0,...|
|0.48259945586790165|            0.47|[296.0,99.0,2.0,3...|
|0.49794648965589006|            0.53|[298.0,99.0,1.0,1...|
| 0.5343089109378827|            0.54|[298.0,101.0,2.0,...|
| 0.5313381713707048|            0.59|[299.0,100.0,1.0,...|
| 0.5540351642939334|            0.63|[299.0,100.0,3.0,...|
| 0.6604607000222611|            0.56|[299.0,102.0,3.0,...|
| 0.6160388782885731|            0.62|[300.0,100.0,3.0,...|
| 0.5830239099545587|            0.67|[301.0,98.0,1.0,2...|
|  0.569034319509155|            0.64|[3

In [40]:

from pyspark.ml.evaluation import RegressionEvaluator
pred_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="Chance of Admit ",metricName="r2")
print("R Squared (R2) on test data =", pred_evaluator.evaluate(predictions))


R Squared (R2) on test data = 0.7790310003191947


implement random forest regression

In [41]:
featureIndexer =VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(output)


In [42]:
featureIndexer

VectorIndexerModel: uid=VectorIndexer_5e49c917e1a9, numFeatures=7, handleInvalid=error

In [43]:
featureIndexer = featureIndexer.transform(output)


In [44]:
new_indexed_data = featureIndexer.select("indexedFeatures", "Chance of Admit ")


In [45]:
training, test = new_indexed_data.randomSplit([0.7, 0.3])


In [46]:
training.show()


+--------------------+----------------+
|     indexedFeatures|Chance of Admit |
+--------------------+----------------+
|[290.0,100.0,1.0,...|            0.47|
|[290.0,104.0,4.0,...|            0.45|
|[293.0,97.0,2.0,2...|            0.64|
|[294.0,93.0,1.0,1...|            0.46|
|[294.0,95.0,1.0,1...|            0.49|
|[295.0,93.0,1.0,2...|            0.46|
|[295.0,99.0,2.0,2...|            0.57|
|[295.0,101.0,2.0,...|            0.69|
|[296.0,95.0,2.0,3...|            0.44|
|[296.0,99.0,2.0,2...|            0.61|
|[296.0,99.0,2.0,3...|            0.47|
|[297.0,96.0,2.0,2...|            0.43|
|[297.0,100.0,1.0,...|            0.52|
|[298.0,98.0,2.0,1...|            0.44|
|[298.0,98.0,2.0,4...|            0.34|
|[298.0,101.0,2.0,...|            0.54|
|[298.0,105.0,3.0,...|            0.69|
|[299.0,94.0,1.0,1...|            0.42|
|[299.0,97.0,3.0,5...|            0.38|
|[299.0,100.0,1.0,...|            0.59|
+--------------------+----------------+
only showing top 20 rows



In [47]:
test.show()


+--------------------+----------------+
|     indexedFeatures|Chance of Admit |
+--------------------+----------------+
|[295.0,96.0,2.0,1...|            0.47|
|[296.0,97.0,2.0,1...|            0.49|
|[296.0,101.0,1.0,...|             0.6|
|[297.0,96.0,2.0,2...|            0.34|
|[297.0,98.0,2.0,2...|            0.59|
|[298.0,92.0,1.0,2...|            0.51|
|[298.0,99.0,1.0,1...|            0.53|
|[298.0,99.0,2.0,4...|            0.46|
|[299.0,96.0,2.0,1...|            0.54|
|[299.0,100.0,3.0,...|            0.63|
|[300.0,99.0,1.0,3...|            0.36|
|[300.0,102.0,3.0,...|            0.63|
|[301.0,106.0,4.0,...|            0.57|
|[302.0,99.0,1.0,2...|            0.57|
|[302.0,101.0,2.0,...|            0.46|
|[302.0,102.0,3.0,...|            0.65|
|[303.0,99.0,3.0,2...|            0.36|
|[303.0,102.0,3.0,...|            0.62|
|[304.0,100.0,4.0,...|            0.42|
|[304.0,102.0,2.0,...|            0.67|
+--------------------+----------------+
only showing top 20 rows



In [49]:
random_forest_reg = RandomForestRegressor(featuresCol="indexedFeatures",labelCol="Chance of Admit " )


In [50]:

# Train model.  This also runs the indexer.
model = random_forest_reg.fit(test)

In [51]:

# Make predictions.
predictions = model.transform(test)

In [52]:
predictions.show()


+--------------------+----------------+-------------------+
|     indexedFeatures|Chance of Admit |         prediction|
+--------------------+----------------+-------------------+
|[295.0,96.0,2.0,1...|            0.47| 0.4832646507205385|
|[296.0,97.0,2.0,1...|            0.49| 0.5140310651831554|
|[296.0,101.0,1.0,...|             0.6| 0.5754369174338423|
|[297.0,96.0,2.0,2...|            0.34| 0.4576176816202474|
|[297.0,98.0,2.0,2...|            0.59| 0.5517651441625573|
|[298.0,92.0,1.0,2...|            0.51| 0.5166464280656407|
|[298.0,99.0,1.0,1...|            0.53| 0.5142078207496118|
|[298.0,99.0,2.0,4...|            0.46| 0.4594420874504236|
|[299.0,96.0,2.0,1...|            0.54| 0.5179517370984926|
|[299.0,100.0,3.0,...|            0.63| 0.5510523778980687|
|[300.0,99.0,1.0,3...|            0.36| 0.4106262022114935|
|[300.0,102.0,3.0,...|            0.63| 0.6436877538517172|
|[301.0,106.0,4.0,...|            0.57| 0.6382477164166485|
|[302.0,99.0,1.0,2...|            0.57|0

In [53]:
evaluator = RegressionEvaluator(labelCol="Chance of Admit ", predictionCol="prediction", metricName="rmse")
print ("Root Mean Squared Error (RMSE) on test data = ",evaluator.evaluate(predictions))


Root Mean Squared Error (RMSE) on test data =  0.045801898517255184


In [54]:

evaluator = RegressionEvaluator(labelCol="Chance of Admit ", predictionCol="prediction", metricName="r2")
print("R Squared (R2) on test data =", evaluator.evaluate(predictions))

R Squared (R2) on test data = 0.9015363909297045


Ridge regression

In [74]:
features.show()

+---------+-----------+-----------------+---+----+----+--------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|
+---------+-----------+-----------------+---+----+----+--------+
|    337.0|      118.0|              4.0|4.5| 4.5|9.65|     1.0|
|    324.0|      107.0|              4.0|4.0| 4.5|8.87|     1.0|
|    316.0|      104.0|              3.0|3.0| 3.5| 8.0|     1.0|
|    322.0|      110.0|              3.0|3.5| 2.5|8.67|     1.0|
|    314.0|      103.0|              2.0|2.0| 3.0|8.21|     0.0|
|    330.0|      115.0|              5.0|4.5| 3.0|9.34|     1.0|
|    321.0|      109.0|              3.0|3.0| 4.0| 8.2|     1.0|
|    308.0|      101.0|              2.0|3.0| 4.0| 7.9|     0.0|
|    302.0|      102.0|              1.0|2.0| 1.5| 8.0|     0.0|
|    323.0|      108.0|              3.0|3.5| 3.0| 8.6|     0.0|
|    325.0|      106.0|              3.0|3.5| 4.0| 8.4|     1.0|
|    327.0|      111.0|              4.0|4.0| 4.5| 9.0|     1.0|
|    328.0|      112.0|  

In [75]:
new_data.show()

+---------+-----------+-----------------+---+----+----+--------+----------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+---------+-----------+-----------------+---+----+----+--------+----------------+
|    337.0|      118.0|              4.0|4.5| 4.5|9.65|     1.0|            0.92|
|    324.0|      107.0|              4.0|4.0| 4.5|8.87|     1.0|            0.76|
|    316.0|      104.0|              3.0|3.0| 3.5| 8.0|     1.0|            0.72|
|    322.0|      110.0|              3.0|3.5| 2.5|8.67|     1.0|             0.8|
|    314.0|      103.0|              2.0|2.0| 3.0|8.21|     0.0|            0.65|
|    330.0|      115.0|              5.0|4.5| 3.0|9.34|     1.0|             0.9|
|    321.0|      109.0|              3.0|3.0| 4.0| 8.2|     1.0|            0.75|
|    308.0|      101.0|              2.0|3.0| 4.0| 7.9|     0.0|            0.68|
|    302.0|      102.0|              1.0|2.0| 1.5| 8.0|     0.0|             0.5|
|    323.0|     

In [68]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

# Assuming you have your SparkSession created and your DataFrame loaded
# df is the DataFrame containing your features and target variable

# Prepare the feature vector using VectorAssembler
assembler = VectorAssembler(inputCols=features.columns[:-1], outputCol="features")
assembled_df = assembler.transform(new_data)
assembled_df= assembled_df.select("features", "Chance of Admit ")

# Split the data into training and testing sets
train_data, test_data = assembled_df.randomSplit([0.7, 0.3], seed=42)
ridge = LinearRegression(featuresCol = 'features', labelCol='Chance of Admit ',regParam=0.5)
ridge_model = ridge.fit(train_df)

# Fit the model to the training data
model = ridge.fit(train_data)

# Make predictions on the testing data
predictions = model.transform(test_data)
predictions.show()


+--------------------+----------------+------------------+
|            features|Chance of Admit |        prediction|
+--------------------+----------------+------------------+
|[293.0,97.0,2.0,2...|            0.64|0.6257333893775563|
|[295.0,96.0,2.0,1...|            0.47| 0.584224428632574|
|[295.0,101.0,2.0,...|            0.69|0.6206631476658084|
|[296.0,95.0,2.0,3...|            0.44|0.6036387546579742|
|[296.0,101.0,1.0,...|             0.6|0.6183155978815931|
|[297.0,96.0,2.0,2...|            0.43|0.6064844282912185|
|[297.0,96.0,2.0,2...|            0.34|0.5993430430100193|
|[298.0,98.0,2.0,1...|            0.44|0.6036134191726621|
|[298.0,99.0,1.0,1...|            0.53|0.6002716331725406|
|[298.0,101.0,2.0,...|            0.54|0.6149935748200208|
|[298.0,105.0,3.0,...|            0.69|0.6978525049746183|
|[299.0,100.0,1.0,...|            0.59|0.6043260506495325|
|[299.0,100.0,2.0,...|            0.68|0.6471915364416143|
|[299.0,100.0,3.0,...|            0.63|0.634004268814807

In [69]:
ridge_model.summary

In [70]:
print("Coefficients: " + str(ridge_model.coefficients))
print("Intercept: " + str(ridge_model.intercept))

Coefficients: [0.0012982356533797496,0.0023934647999104275,0.009799168455848136,0.01017106343245267,0.012409234236611312,0.028571208293879158,0.0163911887439143]
Intercept: -0.3065531968487952


In [72]:
# Evaluate the model's performance
evaluator = RegressionEvaluator(labelCol='Chance of Admit ', predictionCol="prediction", metricName="mse")
mse = evaluator.evaluate(predictions)
print("Mean Squared error on test data =", evaluator.evaluate(predictions))

Mean Squared error on test data = 0.008817643432588099


In [73]:
evaluator = RegressionEvaluator(labelCol='Chance of Admit ', predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print("r2 on test data =", evaluator.evaluate(predictions))

r2 on test data = 0.6135387429675265


Lasso regresssion

In [79]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

# Assuming you have your SparkSession created and your DataFrame loaded
# df is the DataFrame containing your features and target variable

# Prepare the feature vector using VectorAssembler
assembler = VectorAssembler(inputCols=features.columns[:-1], outputCol="features")
assembled_df = assembler.transform(new_data)

# Split the data into training and testing sets
train_data, test_data = assembled_df.randomSplit([0.7, 0.3], seed=42)

# Create an instance of LinearRegression with L1 regularization (Lasso)
lasso = LinearRegression(featuresCol = 'features', labelCol='Chance of Admit ',regParam=0.5, elasticNetParam=1.0)

# Fit the model to the training data
model = lasso.fit(train_data)

# Make predictions on the testing data
predictions = model.transform(test_data)

# Evaluate the model's performance
evaluator = RegressionEvaluator(labelCol="Chance of Admit ", predictionCol="prediction", metricName="mse")
mse = evaluator.evaluate(predictions)


In [80]:
mse = evaluator.evaluate(predictions)
print("Mean Squared error on test data =", evaluator.evaluate(predictions))

Mean Squared error on test data = 0.02300226318940303


In [81]:
# Evaluate the model's performance
evaluator = RegressionEvaluator(labelCol="Chance of Admit ", predictionCol="prediction", metricName="r2")
mse = evaluator.evaluate(predictions)

In [82]:
print("r2 on test data =", evaluator.evaluate(predictions))

r2 on test data = -0.008147314498438973
